# 모듈 1: Agent Development Kit 시작하기

이 모듈에서는 Google의 Agent Development Kit(ADK)의 기본 개념을 소개하고, 첫 번째 에이전트를 만드는 과정을 단계별로 살펴본다. ADK를 활용하면 LLM 기반의 애플리케이션을 보다 체계적으로 개발할 수 있다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다:

- **ADK의 개념**: ADK가 무엇이며 AI 개발에서 왜 중요한지 이해한다.
- **환경 설정**: 에이전트 개발을 위한 Python 환경을 설정한다.
- **에이전트 생성**: 첫 번째 AI 에이전트를 정의하고 생성한다.
- **실행 및 세션 관리**: Runner와 Session을 사용하여 에이전트와 상태 기반 대화를 나눈다.

## ADK란 무엇인가

Google의 Agent Development Kit(ADK)는 AI 에이전트를 구축하기 위한 강력한 프레임워크다. ADK를 사용하면 Gemini와 같은 LLM을 기반으로 도구(Tools)를 사용하고, 기억(Memory)을 유지하며, 복잡한 작업을 수행하는 지능형 에이전트를 쉽게 개발할 수 있다.

## 환경 설정

ADK 사용을 위해 필요한 패키지를 설치한다. `google-adk`는 에이전트 개발을 위한 핵심 라이브러리이며, `python-dotenv`는 환경 변수를 관리하는 데 사용된다.

In [ ]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv

## API 키 설정

Google AI Studio에서 발급받은 API 키를 설정한다. 보안을 위해 API 키를 코드에 직접 노출하지 않고, 환경 변수나 `.env` 파일을 사용하여 관리하는 것이 안전하다. 아래 코드는 `.env` 파일에 API 키를 저장하고 불러오는 과정을 보여준다.

In [1]:
from dotenv import load_dotenv
import os

!echo "GOOGLE_API_KEY=발행한 key 붙혀넣기" >> .env
load_dotenv()

MODEL = "gemini-2.5-flash"

## 첫 번째 에이전트 만들기

이제 `Agent` 클래스를 사용하여 첫 번째 에이전트를 정의한다. 에이전트의 이름, 사용할 모델, 그리고 에이전트가 수행할 역할(instruction)을 설정한다. 여기서는 사용자와 친근하게 인사를 나누는 에이전트를 생성한다.

In [27]:
from google.adk import Agent

# 인사 에이전트 생성
greeting_agent = Agent(
    name="greeting_agent",
    model=MODEL,
    description="사용자와 인사를 나누는 친절한 에이전트",
    instruction="""
    당신은 사용자를 맞이하는 유용한 어시스턴트입니다.
    사용자의 이름을 묻고, 그 이름으로 인사하세요.
    친근하고 대화하듯이 응답하세요.
    """
)

## 에이전트 실행하기 (Runner 및 Session)

ADK에서 에이전트를 실행하기 위해서는 **Runner**와 **Session**이 필요하다. Runner는 에이전트의 실행 환경을 제공하며, Session은 대화의 맥락을 유지하는 역할을 한다. 아래 코드는 `InMemoryRunner`를 초기화하고 새로운 세션을 생성한 뒤, 에이전트에게 첫 번째 메시지를 보내는 과정을 보여준다.

In [28]:
from google.adk.runners import InMemoryRunner

# 1. Runner 생성
runner = InMemoryRunner(agent=greeting_agent)

# 2. 세션 생성 (중요: 키워드 인자 사용)
session = await runner.session_service.create_session(
    app_name="greeting_app", 
    user_id="user_1"
)

print(f"세션이 생성되었습니다. ID: {session.id}")

# 3. 에이전트 실행 (run_debug)
# 생성한 session.id를 전달하여 대화 맥락을 연결한다.
await runner.run_debug("안녕! 반가워.", session_id=session.id)

세션이 생성되었습니다. ID: 2fab3c96-c6fe-43c1-ad46-27a1a5314503

 ### Created new session: 2fab3c96-c6fe-43c1-ad46-27a1a5314503

User > 안녕! 반가워.
greeting_agent > 안녕하세요! 만나서 정말 반가워요. 혹시 성함이 어떻게 되시나요? 😊



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""안녕하세요! 만나서 정말 반가워요. 혹시 성함이 어떻게 되시나요? 😊
 """
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=30,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=30
     ),
   ],
   prompt_token_count=102,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=102
     ),
   ],
   total_token_count=132
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-0.10047307014465331, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-80013a37-422b-4e91-b33a-c89

### 대화 이어가기 (기억 테스트)

생성된 세션 ID를 사용하면 이전 대화의 맥락을 이어갈 수 있다. 에이전트가 앞서 질문한 내용에 대해 답변을 보내고, 에이전트가 이를 기억하고 있는지 확인해 본다.

In [29]:
# 이름을 알려주기
# 동일한 세션 ID를 사용하여 대화를 이어간다.
await runner.run_debug("내 이름은 앨리스야.", session_id=session.id)


 ### Continue session: 2fab3c96-c6fe-43c1-ad46-27a1a5314503

User > 내 이름은 앨리스야.
greeting_agent > 앨리스님, 안녕하세요! 만나 뵙게 되어 정말 기쁩니다. 오늘 하루 즐거운 대화 나누어요! 😊



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""앨리스님, 안녕하세요! 만나 뵙게 되어 정말 기쁩니다. 오늘 하루 즐거운 대화 나누어요! 😊
 """
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=38,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=38
     ),
   ],
   prompt_token_count=140,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=140
     ),
   ],
   total_token_count=178
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-0.1273872224908126, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-0824fa24-032f-4

## 웹 인터페이스로 실행하기

터미널에서 `adk web` 명령어를 실행하면, 웹 브라우저를 통해 에이전트와 대화할 수 있는 인터페이스가 제공된다. 이를 통해 개발한 에이전트를 보다 직관적으로 테스트할 수 있다.

```bash
adk web greeting_agent
```

## 정리

이번 모듈에서는 ADK의 기초적인 사용법을 익혔다. 에이전트를 정의하고, Runner와 Session을 통해 실행 및 대화 상태를 관리하는 방법을 확인했다. 또한, API 키 관리와 같은 기본적인 환경 설정 방법도 함께 다루었다. 다음 모듈에서는 에이전트에 **도구(Tools)**를 연결하여 기능을 확장하는 방법에 대해 알아본다.